In [2]:
import nltk

!pip install sentence-transformers
!pip install autocorrect
!pip install -U spacy
!python -m spacy download en_core_web_sm
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
# spacy.cli.download("en_core_web_sm")
import warnings
warnings.filterwarnings('ignore')

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1283, in rende

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/home/ayush/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1283, in rende

[nltk_data] Downloading package punkt to /home/ayush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ayush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ayush/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ayush/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from autocorrect import Speller
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import spacy
import re
nlp = spacy.load('en_core_web_sm')

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

2022-12-02 22:52:43.418082: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 22:52:43.572963: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-02 22:52:43.619022: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-02 22:52:44.151697: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [4]:
data = pd.read_csv("F2_Claim_Check_Worthiness_train.csv")

In [5]:
# preprocessing funciton created
def preprocessing(df):
    df1 = df.copy()
    df1['white_space_removed'] = 0
    df1['tokenized_data'] = 0
    df1['stopword_removed_data'] = 0
    df1['punct_removed_data'] = 0
    df1['url_removed_data'] = 0
    df1['spelling_checked_data'] = 0
    stop_words = set(stopwords.words("english"))

    # create object for lemmetizer and spelling checking
    # lemmatizer = WordNetLemmatizer()
    spell = Speller(lang='en')


    # iterate over each row of dataset and preprocess data
    for i in range(df1.shape[0]):

        # white space removel
        df1['white_space_removed'][i] = re.sub("\s+", " ", df1.tweet[i])


        # lower casing and tokenization
        lower = df1['white_space_removed'][i].lower()
        tokenized_data = word_tokenize(lower)
        df1['tokenized_data'][i] = tokenized_data
        # print(tokenized_data)


        # remove stopwords
        stopword_removed_data = [x for x in tokenized_data if x not in stop_words]
        df1['stopword_removed_data'][i] = stopword_removed_data 
        # print(stopword_removed_data)


        # punctuation removel
        punct_removed_data = [x for x in stopword_removed_data if x.isalnum()]
        df1['punct_removed_data'][i] = punct_removed_data
        # print(punct_removed_data)


        # remove urls and html tags
        urls = re.findall("https?://[a-zA-Z0-9_\?=\@\/#=.~-]+", " ".join(punct_removed_data))
        url_removed_data = [x for x in punct_removed_data if x not in urls]
        df1['url_removed_data'][i] = url_removed_data
        # print(url_removed_data)
        # print(type(url_removed_data))


        # spelling checking
        spelling_checked_data = [spell(x) for x in url_removed_data]
        df1['spelling_checked_data'][i] = " ".join(spelling_checked_data)
        # print(" ".join(spelling_checked_data))

    return df1

In [6]:
# df = preprocessing(data)

In [7]:
# df = df.rename({'spelling_checked_data': 'preprocessed_text'}, axis=1)
# df.head()

In [8]:
# df_preprocessed = df[['tweet','label','id','preprocessed_text']]

In [9]:
# df_preprocessed.to_csv("preprocessed_train.csv", index=False)

In [13]:
# read train data
df_preprocessed_train = pd.read_csv("preprocessed_train.csv", index_col=False)
df_preprocessed_train.head()

,tweet,label,id,preprocessed_text
0,"India's gift of 100,000 COVID-19 vaccines arri...",0,0,india gift vaccines arrived barbados earlier t...
1,As part of the ongoing nationwide vaccination ...,0,1,part ongoing nationwide vaccination rollout se...
2,"Pleased to receive 50,000 doses of Covid-19 va...",0,2,pleased receive doses vaccines india morning g...
3,Four former presidents have banded together fo...,0,3,four former presidents banded together two nat...
4,WSJ: All three of Russia's main intelligence s...,1,4,wsj three russia main intelligence services sr...


In [14]:
# df_test=pd.read_csv("F2_Claim_Check_Worthiness_test.csv")
# df_test_preprocess=preprocessing(df_test)
# df_test_preprocess.head()

# df_test_preprocess = df_test_preprocess.rename({'spelling_checked_data': 'preprocessed_text'}, axis=1)
# df_test_preprocess.head()
# df_test_preprocess_1 = df_test_preprocess[['tweet','id','preprocessed_text']]
# df_test_preprocess_1.to_csv('preprocessed_test.csv')

In [15]:
# read test data
df_preprocessed_test = pd.read_csv('preprocessed_test.csv')
df_preprocessed_test.head()

,Unnamed: 0,tweet,id,preprocessed_text
0,0,Native American communities have been hit hard...,0,native american communities hit hard thanks in...
1,1,ALERT: CVS Pharmacy is now offering COVID-19 v...,1,alert cvs pharmacy offering vaccines massachus...
2,2,#COVID19 vaccines are an important tool to hel...,2,covid19 vaccines important tool help stop pand...
3,3,Pfizer reports that IRL the vaccine is 97% eff...,3,prize reports irl vaccine 97 effective symptom...
4,4,Vaccines are complex medicines. Europeans can ...,4,vaccines complex medicines european trust vacc...


In [16]:
def pos_tag_extraction(sentence):
    docs = nlp(sentence)
    pos_tags_list=[]
    for token in docs:
        pos_tags_list.append(token.pos_)
    pos_tags = " ".join(pos_tags_list)
    return pos_tags

In [17]:
x = pos_tag_extraction("janak is slleping")

In [18]:
def  dep_feature_extraction(sentence):
    docs = nlp(sentence)
    dep_list=[]
    for token in docs:
        dep_list.append(token.dep_)
    dep_feature = " ".join(dep_list)
    return dep_feature

In [19]:
dep_feature_extraction(x)

'amod compound ROOT'

In [20]:
def get_pos_vector(pos_sentence):
    pos_tags = {'ADJ':0,'ADV':0,'NOUN':0,'PROPN':0,'VERB':0,'NUM':0,'ADP':0,'PRON':0} 
    
    for pos in pos_sentence.split():        
        if pos in pos_tags:
            pos_tags[pos] += 1
            
    return list(pos_tags.values())

In [21]:
get_pos_vector(x)

[0, 0, 0, 1, 1, 0, 0, 0]

In [22]:
def get_count_pos_dep_dict(df):#create set of all unique dependencies in entire data    
    unique_dep = set()
    for row in df['preprocessed_text']:
        pos_sentence = pos_tag_extraction(row).split()
        dep_sentence = dep_feature_extraction(row).split()
        for pos, dep in zip(pos_sentence, dep_sentence):
            unique_dep.add(pos+"_"+dep)
    print(unique_dep)
    return list(unique_dep)

dependency_list = get_count_pos_dep_dict(df_preprocessed_train)

{'ADJ_ROOT', 'ADP_punct', 'PROPN_intj', 'PROPN_dep', 'CCONJ_preconj', 'SCONJ_mark', 'ADJ_conj', 'SCONJ_advmod', 'PROPN_punct', 'VERB_punct', 'NOUN_dobj', 'ADP_prep', 'ADP_advmod', 'NUM_dobj', 'ADJ_punct', 'ADP_nmod', 'PART_compound', 'ADJ_appos', 'ADV_nsubj', 'NUM_meta', 'PROPN_attr', 'NUM_punct', 'VERB_attr', 'AUX_nsubj', 'NOUN_pobj', 'CCONJ_det', 'VERB_prep', 'X_ROOT', 'NOUN_amod', 'INTJ_amod', 'CCONJ_advmod', 'NOUN_attr', 'NUM_nmod', 'NUM_dep', 'NOUN_dative', 'ADP_oprd', 'NOUN_det', 'AUX_aux', 'NOUN_xcomp', 'DET_poss', 'PROPN_acl', 'ADP_det', 'ADJ_prep', 'NOUN_ROOT', 'NUM_dative', 'VERB_dobj', 'VERB_nsubj', 'ADV_advmod', 'ADJ_acl', 'VERB_poss', 'PRON_dative', 'PROPN_prep', 'ADJ_xcomp', 'ADP_compound', 'ADV_pcomp', 'NUM_npadvmod', 'X_compound', 'NUM_pobj', 'ADJ_nummod', 'ADV_dobj', 'ADJ_npadvmod', 'DET_det', 'ADV_nmod', 'ADV_cc', 'PART_neg', 'NOUN_quantmod', 'VERB_acomp', 'X_dep', 'VERB_appos', 'INTJ_advmod', 'NOUN_relcl', 'PRON_nmod', 'ADP_intj', 'ADJ_oprd', 'PROPN_pobj', 'PRON_npad

In [17]:
def get_pos_dep_dict(df):#create set of all unique dependencies in entire data    
    unique_dep = {}
    for row in df['preprocessed_text']:
        pos_sentence = pos_tag_extraction(row).split()
        dep_sentence = dep_feature_extraction(row).split()
        for pos, dep in zip(pos_sentence, dep_sentence):
            if (pos+"_"+dep) not in unique_dep:
                unique_dep[(pos+"_"+dep)] = 1
            else:
                unique_dep[(pos+"_"+dep)] += 1            
    return unique_dep

dependency_dict = get_pos_dep_dict(df_preprocessed_train)

In [18]:
from collections import Counter

common_dep = Counter(dependency_dict).most_common(15)
important_dep = []

for dep in common_dep:
    important_dep.append(dep[0])
    
print(important_dep)

['NOUN_compound', 'ADJ_amod', 'NOUN_dobj', 'PROPN_compound', 'NOUN_nsubj', 'VERB_ROOT', 'VERB_ccomp', 'NUM_nummod', 'ADV_advmod', 'PROPN_nsubj', 'VERB_acl', 'NOUN_nmod', 'NOUN_npadvmod', 'VERB_amod', 'VERB_advcl']


In [19]:
len(important_dep)

15

In [20]:
def get_dep_vector(dep_sentence): 
    pos_tags = {'ADJ':0,'ADV':0,'NOUN':0,'PROPN':0,'NUM':0,} 
    pos_sentence = pos_tag_extraction(dep_sentence).split(" ")
    dep_sentence = dep_feature_extraction(dep_sentence).split(' ')
    sentence_dep_dict = dict.fromkeys(important_dep, 0)
    for pos, dep in zip(pos_sentence, dep_sentence):
      if pos in pos_tags:
        if (pos+"_"+dep) in sentence_dep_dict and (pos+"_"+dep) in important_dep:
            sentence_dep_dict[(pos+"_"+dep)] += 1
    return list(sentence_dep_dict.values())

In [30]:
def scale_df(df):
  scaler = MinMaxScaler()
  temp = df.copy()
  transformed_data = pd.DataFrame(scaler.fit_transform(temp))
  return transformed_data

In [33]:
def apply_pca(df, component=10):
  pca = PCA(n_components=component)
  pca_df = pd.DataFrame(pca.fit_transform(df))
  return pca_df

In [70]:
def get_sentence_embedding(df):
  model = SentenceTransformer('sentence-transformers/nli-roberta-large')
  embeddings = []
  for i in range(len(df)):
    embeddings.append(model.encode(df['preprocessed_text'][i]))
  sentence_embedding = pd.DataFrame(embeddings)
  return sentence_embedding

In [66]:
def pipeline(df):
  # get pos and dep feature
  final_feature = []
  for row in df['preprocessed_text']:
      temp = []
      temp.extend(get_pos_vector(row))
      temp.extend(get_dep_vector(row))
      final_feature.append(temp)
  df_features = pd.DataFrame(final_feature)

  # sentence embeddings
  sentence_embedding = get_sentence_embedding(df)

  # merge data
  final_features = pd.concat([df_features, sentence_embedding], axis=1)
  final_features.columns = [list(range(len(df_features.columns) + len(sentence_embedding.columns)))]

  # scale data
  scaled_df = scale_df(final_features)

  # apply PCA
  pca_df = apply_pca(scaled_df)

  # return dataframe
  return pca_df

In [71]:
df_preprocessed_train.head()

,tweet,label,id,preprocessed_text
0,"India's gift of 100,000 COVID-19 vaccines arri...",0,0,india gift vaccines arrived barbados earlier t...
1,As part of the ongoing nationwide vaccination ...,0,1,part ongoing nationwide vaccination rollout se...
2,"Pleased to receive 50,000 doses of Covid-19 va...",0,2,pleased receive doses vaccines india morning g...
3,Four former presidents have banded together fo...,0,3,four former presidents banded together two nat...
4,WSJ: All three of Russia's main intelligence s...,1,4,wsj three russia main intelligence services sr...


In [72]:
final_df_train = pipeline(df_preprocessed_train)
final_df_train.head()

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/673 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

,0,1,2,3,4,5,6,7,8,9
0,-1.989628,-0.515625,0.243458,-1.467034,-1.544134,0.634842,-0.193154,0.334937,1.193011,-0.953150
1,-0.857630,-0.191991,-0.165533,-1.778217,0.352471,-0.359957,0.535088,1.217062,-0.937238,0.869700
2,-1.997131,0.582907,0.291165,-1.695340,-0.579745,0.248842,-0.951660,-0.208313,0.266242,-0.248287
3,-0.667458,-0.577157,-0.639767,-0.742149,-0.241350,-0.169785,-0.006982,0.108300,-1.067826,0.225590
4,2.345194,0.446420,-0.306944,0.877235,0.313330,0.754666,-1.232776,0.833002,0.772805,-0.073914


In [78]:
df_preprocessed_test = pipeline(df_preprocessed_test)
df_preprocessed_test.head()

,0,1,2,3,4,5,6,7,8,9
0,-0.232680,-0.171172,0.443812,-0.133385,-1.299774,1.926127,-1.336011,0.403901,-0.047194,-0.104638
1,-1.171152,-0.580137,-0.259902,1.490994,-1.356849,-0.354300,-2.129738,-1.731123,-0.375972,1.252226
2,-1.764543,0.167640,1.627765,-1.060465,-1.109390,-0.351949,0.267373,1.127492,-0.447704,1.248427
3,-2.019849,1.130434,2.818671,-2.030438,1.223651,1.547851,0.397922,0.933843,-0.119781,-1.216461
4,-2.309898,2.070880,-0.807743,-0.922071,0.266529,0.535223,-0.546703,-0.091210,-1.038256,-0.608364


In [1]:
# train test split
y = df_preprocessed_train.label
X_train, X_test, y_train, y_test = train_test_split(final_df_train, y, train_size=0.8, random_state=40, stratify=y)

NameError: ignored

In [ ]:
# train data on SVM model
model = SVC(kernel='poly', degree=121, class_weight='balanced')
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print("train acc :", accuracy_score(y_train, y_pred_train))
print("test acc:", accuracy_score(y_test, y_pred_test))

In [84]:
y_pred = model.predict(final_df_test)
len(y_pred)

195

In [85]:
ID = [i for i in range(len(y_pred))]
output_df = pd.DataFrame(list(zip(y_pred, ID)), columns=['label', 'id'])
output_df.to_csv("output2.csv", index=False)

In [26]:
import re
output = re.sub(r'\d+', '', '123hello 456world 123bye')
print (output)  # 'hello world'

hello world bye
